Chapitre 10 - Session utilisations et Insertions SQL
===

## Bibliographie

- [Formulaires avec WTForms](https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-iii-web-forms)
- [How NOT to Store Passwords](https://www.youtube.com/watch?v=8ZtInClXe1Q)

In [2]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/8ZtInClXe1Q" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

## Créer des comptes

La création de comptes est une des tâches les plus courantes dans le cadre d'un développement d'application. C'est aussi une des plus dangereuses : vous êtes responsables de la sécurité de vos utilisateurs et de leurs données. On apprendra donc ici à sécuriser un maximum nos données sensibles.

### Gérer des mots de passe

Pour enregistrer un mot de passe, on va *à minima* encrypter ce mot de passe. On utilise d'ailleurs à mauvais escient ce terme car ce que l'on va calculer réellement est un hash. Contrairement au cryptage, le hash n'est pas réversible. Si quelque chose est hashé en "qqewretet", la seule manière de trouver l'original est de "brute-forcer" ce dernier, c'est-à-dire de générer tous les mots de passe possibles (ce qui est, disons-le, plutôt difficile).

L'autre manière de casser un Hash, c'est d'avoir la malchance d'avoir l'algorithme de hashage de cassé. On en trouve trois actuellement recommandés (*cf.* *[
About Secure Password Hashing](http://security.blogoverflow.com/2013/09/about-secure-password-hashing/)*).

Pour Flask, Werkzeug (un outil qui permet à Flask de fonctionner) fournit deux fonctions : `generate_password_hash` et `check_password_hash`.

#### Constante de sécurité

Pour notre application, nous allons intégrer un nouveau module pour stocker nos constantes. Cela permettra de gérer les configurations générales plus proprement : créons donc `gazetteer/constantes.py` où nous stockons un "secret" qui permettra à Flask d'effectuer des transactions sécurisées :

In [ ]:
%pycat cours-flask/exemple16/gazetteer/constantes.py

Le sel devant être unique et non devinable, on rajoute un avertissement au cas où la personne mettant ce site en fonctionnement n'avait pas connaissance de ce changement de configuration.

#### Méthode propre

On va maintenant créer une fonction pour vérifier une connexion. Rappel de notre classe `User`:

```python
class User(db.Model):
    user_id = db.Column(db.Integer, unique=True, nullable=False, primary_key=True, autoincrement=True)
    user_nom = db.Column(db.Text, nullable=False)
    user_login = db.Column(db.String(45), nullable=False, unique=True)
    user_email = db.Column(db.Text, nullable=False)
    user_password = db.Column(db.String(100), nullable=False)
```

Pour vérifier la validité de l'identification de l'utilisateur, on va retrouver l'utilisateur, comparer le hash généré du mot de passe avec celui enregistré :

```python
def identification(login, motdepasse):
    """ Identifie un utilisateur. Si cela fonctionne, renvoie les données de l'utilisateur.
    
    :param login: Login de l'utilisateur
    :param motdepasse: Mot de passe envoyé par l'utilisateur
    :returns: Si réussite, données de l'utilisateur. Sinon None
    :rtype: User or None
    """
    utilisateur = User.query.filter(User.user_login == login).first()
    if utilisateur and check_password_hash(utilisateur.user_password, motdepasse):
        return utilisateur
    return None
```

Pour que cette fonction soit facile à retrouver, on va l'enregistrer sous la responsabilité de la classe `User` :

```python
class User(db.Model):
    user_id = db.Column(db.Integer, unique=True, nullable=False, primary_key=True, autoincrement=True)
    user_nom = db.Column(db.Text, nullable=False)
    user_login = db.Column(db.String(45), nullable=False, unique=True)
    user_email = db.Column(db.Text, nullable=False)
    user_password = db.Column(db.String(64), nullable=False)
    
    @staticmethod
    def identification(login, motdepasse):
        """ Identifie un utilisateur. Si cela fonctionne, renvoie les données de l'utilisateur.

        :param login: Login de l'utilisateur
        :param motdepasse: Mot de passe envoyé par l'utilisateur
        :returns: Si réussite, données de l'utilisateur. Sinon None
        :rtype: User or None
        """
        utilisateur = User.query.filter(User.user_login == login).first()
        if utilisateur and check_password_hash(utilisateur.user_password, motdepasse):
            return utilisateur
        return None
```

On précède notre fonction de `@staticmethod` pour mettre d'appeler cette fonction ainsi :

```python
utilisateur = User.identification(login, motdepasse)
```

#### Premier insert

On va maintenant effectuer notre premier insert ! Et oui, il va falloir créer des comptes. Pour créer un compte, il nous faudra aussi une fonction de création de comptes.

Pour créer un enregistrement ou une mise à jour, SQL et SQLAlchemy fonctionnent par session de changement, un peu comme git : on crée un ensemble de modifications, on les stocke pour l'envoi (`db.session.add(lachoseaenvoyer)`) et on envoie à MySQL (`db.session.commit()`). Prenez le temps de bien lire le contenu qui suit :

```python
class User(db.model):
    # ...
    @staticmethod
    def creer(login, email, nom, motdepasse):
        """ Crée un compte utilisateur-rice. Retourne un tuple (booléen, User ou liste).
        Si il y a une erreur, la fonction renvoie False suivi d'une liste d'erreur
        Sinon, elle renvoie True suivi de la donnée enregistrée

        :param login: Login de l'utilisateur-rice
        :param email: Email de l'utilisateur-rice
        :param nom: Nom de l'utilisateur-rice
        :param motdepasse: Mot de passe de l'utilisateur-rice (Minimum 6 caractères)

        """
        erreurs = []
        if not login:
            erreurs.append("Le login fourni est vide")
        if not email:
            erreurs.append("L'email fourni est vide")
        if not nom:
            erreurs.append("Le nom fourni est vide")
        if not motdepasse or len(motdepasse) < 6:
            erreurs.append("Le mot de passe fourni est vide ou trop court")

        # On vérifie que personne n'a utilisé cet email ou ce login
        uniques = User.query.filter(
            db.or_(User.user_email == email, User.user_login == login)
        ).count()
        if uniques > 0:
            erreurs.append("L'email ou le login sont déjà inscrits dans notre base de données")

        # Si on a au moins une erreur
        if len(erreurs) > 0:
            return False, erreurs

        # On crée un utilisateur
        utilisateur = User(
            user_nom=nom,
            user_login=login,
            user_email=email,
            user_password=generate_password_hash(motdepasse)
        )

        try:
            # On l'ajoute au transport vers la base de données
            db.session.add(utilisateur)
            # On envoie le paquet
            db.session.commit()

            # On renvoie l'utilisateur
            return True, utilisateur
        except Exception as erreur:
            return False, [str(erreur)]
```

### Formulaire d'inscription

On ajoute ensuite la route pour s'inscrire (ainsi qu'un lien dans le menu !) :

```python
from flask import flash, redirect, request


@app.route("/register", methods=["GET", "POST"])
def inscription():
    """ Route gérant les inscriptions
    """
    # Si on est en POST, cela veut dire que le formulaire a été envoyé
    if request.method == "POST":
        statut, donnees = User.creer(
            login=request.form.get("login", None),
            email=request.form.get("email", None),
            nom=request.form.get("nom", None),
            motdepasse=request.form.get("motdepasse", None)
        )
        if statut is True:
            flash("Enregistrement effectué. Identifiez-vous maintenant", "success")
            return redirect("/")
        else:
            flash("Les erreurs suivantes ont été rencontrées : " + ",".join(donnees), "error")
            return render_template("pages/inscription.html")
    else:
        return render_template("pages/inscription.html")
```

#### On remarque: 

- Dans @app.route, on a ajouté `methods=["GET", "POST"]` : cela signifie que la page est disponible à la fois pour GET et POST
- On peut vérifier la méthode utilisée en vérifiant la valeur de `request.method`
- On peut utiliser la fonction `flask.flash` pour afficher des messages d'erreurs (Regardez le `container.html` qui a été modifié pour afficher celles-ci). Les erreurs peuvent être accompagnées de catégorie.
- On peut rediriger vers une url précise via `flask.redirect`
- On a préféré séparer la logique de la connexion (dans modeles/utilisateur.py) plutôt que dans les routes. La route `inscription` ne sert qu'à appeler les différentes fonctions.

## Se connecter

Pour gérer les utilisateurs, Flask possède un plugin `flask-login` que nous allons installer :

In [ ]:
!pip install flask-login

### Configuration

Flask-Login s'installe de la même manière que Flask SQLAlchemy :

```python
from flask_login import LoginManager
app = Flask(...)
# ...
login = LoginManager(app)
```

On ajoute donc ces lignes à notre `app.py`.

Une fois cela fait, on va configurer notre objet SQLAlchemy User pour qu'il puisse être compris par Flask Login, il va falloir rajouter quatre fonctions :
- `is_authenticated` : une propriété, elle retournera True si l'utilisateur-rice est connecté-e, False sinon
- `is_active` : une propriété, elle retournera True si l'utilisateur-rice est actif-ve, False sinon. 
    - Le caractère actif des membres de notre site n'est pas utilisé, pourtant, il faudra remplir cette case. On renverra donc True
- `is_anonymous` : une propriété qui retournera False pour les utilisateur-rices non anonymes.
- `get_id(identifier)` : une fonction qui retournera l'utilisateur pour l'ID donné

**Mais** les choses sont bien faites, Flask Login propose un outil pour ne pas avoir à coder cela soit même : `flask_login.UserMixin` :

```python
from werkzeug.security import generate_password_hash, check_password_hash
from flask_login import UserMixin

from .. app import db

class User(UserMixin, db.Model):
    # ...
```

**Explications** : en ajoutant `UserMixin` à `db.Model`, on donne à python l'information que User est à la fois un `UserMixin` et un `db.Model`

**Malheureusement**, le `get_id` par défaut de UserMixin n'est pas complètement compatible avec notre `User`, car il a besoin d'une propriété `.id` pour que `get_id()` fonctionne. On va donc écrire par-dessus:

```python
class User(UserMixin, db.Model):
    # ...
    def get_id(self):
        """ Retourne l'id de l'objet actuellement utilisé 
        
        :returns: ID de l'utilisateur
        :rtype: int
        """
        return self.user_id
```

Ici, `self` prend la valeur de l'utilisateur courant. Par exemple :

```python
Laurel = User(user_id=1)
Hardy = User(user_id=2)

print(Laurel.get_id())
>>> 1
print(Hardy.get_id())
>>> 2
```

Il va quand même falloir définir nous-mêmes une fonction qui permettra de récupérer un utilisateur en fonction de son identifiant (toujours dans `utilisateur.py`) :

```python
from app import login

# ...
@login.user_loader
def trouver_utilisateur_via_id(id):
    return User.query.get(int(id))
```



### Formulaire de connexion

On ajoutera donc un nouveau formulaire à notre application ainsi qu'un nouveau menu dans le container principal :

```html
{% extends "conteneur.html" %}

{% block titre %}| Connexion{%endblock%}

{% block corps %}

<h1>Inscription</h1>
<form class="form" method="POST" action="{{url_for("connexion")}}">
  <div class="form-group row">
    <label for="register-login" class="col-sm-2 col-form-label">Login</label>
    <div class="col-sm-10">
      <input type="text" class="form-control" id="register-login" name="login" placeholder="Nom d'utilisateur pour se connecter">
    </div>
  </div>
  <div class="form-group row">
    <label for="register-password" class="col-sm-2 col-form-label">Password</label>
    <div class="col-sm-10">
      <input type="password" class="form-control" id="register-password" placeholder="Mot de passe" name="motdepasse">
    </div>
  </div>
  <div>
    <button type="submit" class="btn btn-primary">Connexion</button>
    <a href="{{url_for("inscription")}}" class="btn btn-secondary">Inscription</a>
  </div>
</form>
{% endblock %}
```

et la route qui correspondra 

```python
from flask_login import login_user, current_user
from .app import login


@app.route("/connexion", methods=["POST", "GET"])
def connexion():
    """ Route gérant les connexions
    """
    if current_user.is_authenticated is True:
        flash("Vous êtes déjà connecté-e", "info")
        return redirect("/")
    # Si on est en POST, cela veut dire que le formulaire a été envoyé
    if request.method == "POST":
        utilisateur = User.identification(
            login=request.form.get("login", None),
            motdepasse=request.form.get("motdepasse", None)
        )
        if utilisateur:
            flash("Connexion effectuée", "success")
            login_user(utilisateur)
            return redirect("/")
        else:
            flash("Les identifiants n'ont pas été reconnus", "error")

    return render_template("pages/connexion.html")
        
login.login_view = 'connexion'
```

##### On remarquera
- Pour obtenir l'utilisateur courant, on appelle `flask_login.current_user`
- Pour valider l'authentification, on passer à la fonction `flask_login.login_user` la variable comportant l'utilisateur
- On marque la route définie comme celle de connexion

### Deconnexion

La route de déconnexion est aussi assez simple : on va juste utiliser la fonction `flask_login.logout_user` :

```python
from flask_login import logout_user, current_user

@app.route("/deconnexion", methods=["POST", "GET"])
def deconnexion():
    if current_user.is_authenticated is True:
        logout_user()
    flash("Vous êtes déconnecté-e", "info")
    return redirect("/")
```

### Afficher l'utilisateur courant dans les templates

Pour afficher l'utilisateur courant dans les templates, on utilise de même la variable `{{ current_user }}`: 

```html
<ul class="navbar-nav mr-auto">
    {% if not current_user.is_authenticated %}
      <li class="nav-item">
        <a class="nav-link" href="{{url_for("inscription")}}">Inscription</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" href="{{url_for("connexion")}}">Connexion</a>
      </li>
    {% else %}
      <li class="nav-item">
        <a class="nav-link" href="{{url_for("deconnexion")}}">Déconnexion ({{current_user.user_nom}})</a>
      </li>
    {% endif %}
```

#### Exemple 16

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-flask/exemple16
python run.py
```

#### Exercice

Écrire une page pour créer un nouveau lieu seulement quand on est connecté. Pour limiter la capacité à accéder à une page, on précède la route de `flask_login.login_required` :

```python
from flask_login import login_required

@login_required
def route()
```